In [ ]:
import tweepy
import time
import csv
from datetime import datetime
import pytz
import os

# Claves de autenticación
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ACCESS_TOKEN_SECRET = os.getenv("ACCESS_TOKEN_SECRET")
BEARER_TOKEN = os.getenv("BEARER_TOKEN")

Client = tweepy.Client(bearer_token=BEARER_TOKEN, consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET, access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET)

# Lista de usuarios por sus nombres de usuario
usuarios = ['pereira_sa6822']
# usuarios = [
#     'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
#     'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
#     'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
#     'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
#     'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
#     'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
#     'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
#     'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
#     'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
#     'CiudaDanaMtySur']

# Función para obtener los IDs de los usuarios y guardarlos en un CSV
def guardar_usuarios_ids(usuarios):
    usuarios_ids = []
    for username in usuarios:
        try:
            user = Client.get_user(username='periera_sa6822')
            if user:
                usuarios_ids.append({'username': username, 'user_id': user.id})
                print(f"Usuario {username} obtenido correctamente.")
            else:
                print(f"No se encontró información para el usuario {username}.")
        except tweepy.TweepyException as e:
            print(f"Error al obtener el ID para el usuario {username}: {e}")

    # Guardar los usuarios y sus IDs en un archivo CSV
    with open('usuarios_ids.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['username', 'user_id']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for usuario in usuarios_ids:
            writer.writerow(usuario)
    print("Usuarios y IDs guardados en 'usuarios_ids.csv'")

# Función para obtener los tweets de un usuario limitado a partir de 2024
def get_user_tweets(user_id, count):
    try:
        # Limitar los tweets a partir de enero de 2024
        start_time = datetime(2024, 1, 1)
        
        # Obtener los tweets del usuario con los campos necesarios
        tweets = Client.user_timeline(user_id=user_id, count=count, tweet_mode='extended')
        tweets_filtrados = [
            tweet for tweet in tweets if tweet.created_at >= start_time
        ]
        
        # Verificar si se obtuvieron tweets correctamente
        if tweets_filtrados:
            print(f"Tweets obtenidos correctamente para el usuario con ID {user_id}")
            return tweets_filtrados
        else:
            print(f"No se encontraron tweets para el usuario con ID {user_id}")
            return None

    except tweepy.TweepyException as e:
        print(f"Error al obtener tweets para el usuario con ID {user_id}: {e}")
        return None

# Manejo de Rate Limits y revisión de éxito
def get_tweets_multiple_users(csv_filename, count):
    # Definir la zona horaria de Monterrey
    zona_horaria_monterrey = pytz.timezone('America/Monterrey')
    
    tweets_por_usuario = {}
    
    # Leer los usuarios y sus IDs desde el archivo CSV
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            user_id = row['user_id']
            username = row['username']
            try:
                tweets = get_user_tweets(user_id, count)
                if tweets:
                    # Guardar 'created_at', 'public_metrics', 'entities' y 'geo' de cada tweet
                    tweets_por_usuario[username] = [
                        {
                            'username': username,
                            'created_at': tweet.created_at,
                            'retweet_count': tweet.retweet_count,
                            'reply_count': tweet.favorite_count,  # No existe reply_count en tweepy v1
                            'like_count': tweet.favorite_count,
                            'quote_count': 0,  # La API v1 no soporta contar quotes directamente
                            'entities': tweet.entities,
                            'geo': tweet.geo
                        }
                        for tweet in tweets
                    ]
                    print(f"Datos procesados correctamente para el usuario {username}")
                else:
                    print(f"Sin datos para procesar para el usuario {username}")

            except tweepy.RateLimitError:
                # Obtener la hora actual en la zona horaria de Monterrey cuando se detecta el rate limit
                hora_actual_monterrey = datetime.now(zona_horaria_monterrey).strftime("%Y-%m-%d %H:%M:%S")
                print(f"Rate limit alcanzado a las {hora_actual_monterrey}. Esperando 15 minutos...")
                time.sleep(900)  # Pausa de 15 minutos para respetar el rate limit

            except Exception as e:
                print(f"Error inesperado para el usuario {username}: {e}")
    
    # Guardar los resultados en un archivo CSV
    with open('tweets_usuarios.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['username', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'entities', 'geo']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for usuario, tweets in tweets_por_usuario.items():
            for tweet in tweets:
                writer.writerow({
                    'username': usuario,
                    'created_at': tweet['created_at'],
                    'retweet_count': tweet['retweet_count'],
                    'reply_count': tweet['reply_count'],
                    'like_count': tweet['like_count'],
                    'quote_count': tweet['quote_count'],
                    'entities': tweet['entities'],
                    'geo': tweet['geo']
                })

    print("Tweets guardados en 'tweets_usuarios.csv'")

# Guardar los usuarios y sus IDs en un archivo CSV
guardar_usuarios_ids(usuarios)

# # Obtener los tweets de los usuarios a partir del archivo CSV generado
# get_tweets_multiple_users('usuarios_ids.csv', 50)

Error al obtener el ID para el usuario pereira_sa6822: 401 Unauthorized
Unauthorized
Usuarios y IDs guardados en 'usuarios_ids.csv'


In [2]:
import tweepy
import time
import csv
from datetime import datetime
import pytz
import os
import requests
import base64

# Claves de autenticación
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# Función para obtener el Bearer Token mediante OAuth 2.0 App-only
def obtener_bearer_token():
    url = "https://api.twitter.com/oauth2/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode(),
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"Error al obtener el Bearer Token: {response.status_code} - {response.text}")

# Obtener el Bearer Token
token = obtener_bearer_token()

# Autenticación mediante Bearer Token
client = tweepy.Client(bearer_token=token)

# Lista de usuarios por sus nombres de usuario
usuarios = ['pereira_sa6822']
# usuarios = [
#     'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
#     'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
#     'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
#     'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
#     'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
#     'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
#     'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
#     'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
#     'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
#     'CiudaDanaMtySur']

# Función para obtener los IDs de los usuarios y guardarlos en un CSV
def guardar_usuarios_ids(usuarios):
    usuarios_ids = []
    for username in usuarios:
        try:
            user = client.get_user(username=username, user_fields=['id'])
            if user.data:
                usuarios_ids.append({'username': username, 'user_id': user.data.id})
                print(f"Usuario {username} obtenido correctamente.")
            else:
                print(f"No se encontró información para el usuario {username}.")
        except tweepy.TweepyException as e:
            print(f"Error al obtener el ID para el usuario {username}: {e}")

    # Guardar los usuarios y sus IDs en un archivo CSV
    with open('usuarios_ids.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['username', 'user_id']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for usuario in usuarios_ids:
            writer.writerow(usuario)
    print("Usuarios y IDs guardados en 'usuarios_ids.csv'")

# Función para obtener los tweets de un usuario limitado a partir de 2024
def get_user_tweets(user_id, count):
    try:
        # Limitar los tweets a partir de enero de 2024
        start_time = datetime(2024, 1, 1).isoformat() + 'Z'
        
        # Obtener los tweets del usuario con los campos necesarios
        tweets = client.get_users_tweets(
            id=user_id,
            max_results=count,
            start_time=start_time,
            tweet_fields=['created_at', 'public_metrics', 'entities', 'geo']
        )
        
        # Verificar si se obtuvieron tweets correctamente
        if tweets.data:
            print(f"Tweets obtenidos correctamente para el usuario con ID {user_id}")
            return tweets.data
        else:
            print(f"No se encontraron tweets para el usuario con ID {user_id}")
            return None

    except tweepy.TweepyException as e:
        print(f"Error al obtener tweets para el usuario con ID {user_id}: {e}")
        return None

# Manejo de Rate Limits y revisión de éxito
def get_tweets_multiple_users(csv_filename, count):
    # Definir la zona horaria de Monterrey
    zona_horaria_monterrey = pytz.timezone('America/Monterrey')
    
    tweets_por_usuario = {}
    
    # Leer los usuarios y sus IDs desde el archivo CSV
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            user_id = row['user_id']
            username = row['username']
            try:
                tweets = get_user_tweets(user_id, count)
                if tweets:
                    # Guardar 'created_at', 'public_metrics', 'entities' y 'geo' de cada tweet
                    tweets_por_usuario[username] = [
                        {
                            'username': username,
                            'created_at': tweet.created_at,
                            'retweet_count': tweet.public_metrics['retweet_count'],
                            'reply_count': tweet.public_metrics['reply_count'],
                            'like_count': tweet.public_metrics['like_count'],
                            'quote_count': tweet.public_metrics['quote_count'],
                            'entities': tweet.entities,
                            'geo': tweet.geo
                        }
                        for tweet in tweets
                    ]
                    print(f"Datos procesados correctamente para el usuario {username}")
                else:
                    print(f"Sin datos para procesar para el usuario {username}")

            except tweepy.TooManyRequests:
                # Obtener la hora actual en la zona horaria de Monterrey cuando se detecta el rate limit
                hora_actual_monterrey = datetime.now(zona_horaria_monterrey).strftime("%Y-%m-%d %H:%M:%S")
                print(f"Rate limit alcanzado a las {hora_actual_monterrey}. Esperando 15 minutos...")
                time.sleep(900)  # Pausa de 15 minutos para respetar el rate limit

            except Exception as e:
                print(f"Error inesperado para el usuario {username}: {e}")
    
    # Guardar los resultados en un archivo CSV
    with open('tweets_usuarios.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['username', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'entities', 'geo']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for usuario, tweets in tweets_por_usuario.items():
            for tweet in tweets:
                writer.writerow({
                    'username': usuario,
                    'created_at': tweet['created_at'],
                    'retweet_count': tweet['retweet_count'],
                    'reply_count': tweet['reply_count'],
                    'like_count': tweet['like_count'],
                    'quote_count': tweet['quote_count'],
                    'entities': tweet['entities'],
                    'geo': tweet['geo']
                })

    print("Tweets guardados en 'tweets_usuarios.csv'")

# Guardar los usuarios y sus IDs en un archivo CSV
guardar_usuarios_ids(usuarios)

# Obtener los tweets de los usuarios a partir del archivo CSV generado
get_tweets_multiple_users('usuarios_ids.csv', 50)

Exception: Error al obtener el Bearer Token: 403 - {"errors":[{"code":99,"message":"Unable to verify your credentials","label":"authenticity_token_error"}]}